# 0: Libraries and Functionalities

In [ ]:
#import all the needed libraries and initialize them

import pandas as pd
import numpy as np
import math
import librosa as lr
import librosa.display
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

#playing audio
import IPython.display as ipd

In [ ]:
# make list variables for the needed files these are like global variables
audio_files = []
file_ids = []

In [ ]:
class StopWatch():

    def __init__(self):
        self.start_time = time.time()

    def give(self):
        time_diff = round(time.time() - self.start_time)
        hour = str(time_diff // 3600).zfill(2)
        minute = str((time_diff % 3600) // 60).zfill(2)
        second = str(time_diff % 60).zfill(2)  # Same as time_diff - (minutes * 60)
        
        return f'[{hour}:{minute}:{second}]'


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Change this to your directory
%cd 'drive/My Drive/Music + AI Project/01 - [NN1] LSTM Generation'
#%cd 'Music + AI Project/01 - [NN1] LSTM Generation'

/content/drive/My Drive/Music + AI Project/01 - [NN1] LSTM Generation


In [ ]:
global_sr = int(44100/2)

# 1: Setting up the files in Google File System (GFS)

Adding all the files and their IDs to the google file system so it is easy to access without drive permissions. New files can be added to this list with the format (file-share-id, file-name)

In [ ]:
#have the file sharing IDs
#Russian Circles file IDs
russian_circles = []
russian_circles.append({ "file_id": "fake-id", "file_name" : "full_albums_wav/RussianCircles_BloodYear.wav"})

file_ids = russian_circles

In [ ]:
#have the file sharing IDs
#Russian Circles file IDs
bernard_purdie = []
bernard_purdie.append({ "file_id": "fake-id", "file_name" : "full_albums_wav/BernardPurdie_MasterDrummers_V1.wav"})
bernard_purdie.append({ "file_id": "fake-id", "file_name" : "full_albums_wav/BernardPurdie_MasterDrummers_V2.wav"})

file_ids = bernard_purdie

This is the actual code which gets the file from the drive to GFS for easy read, no need to run this if they exist in the 'content' folder.

In [ ]:
# a functino that can get these files on temp folder
def getfile(file_id):
    !gdown --id $file_id
    
#loop on all files IDs to read them
for i,id in enumerate(file_ids):
    getfile(id[0])

# 2: Converting the Files into numpy array for the system

**Inputs**:

Variable name | Description
--- | ---
chunck_seconds | amount of sound data each chuck should have in it
overlap_seconds | how much overlap is expected with the previous chuck (will consider padding for the first chunk)
sr | sampling rate while reading audio, also how many features per chunk (default: 44100)
hop_seconds | how much seconds to hop for the next chuck (default: starts at the end of previous chunk, considering overlap)

---

**Outputs**:

- Prints the total number of features created as (no. of features/chunks, samples in each chunk)
- saves the features into a file: 'data/feature_data'


In [ ]:
#Folder from where files need to be read
#can be converted to glob later
music_folder = ""


#specifics for the audio extraction
chunck_seconds = 10
overlap_seconds = 1
sr = global_sr
#hop_seconds = 8
hop_seconds = chunck_seconds - overlap_seconds          #default value

#concerting inputs to samples
chunk = sr * chunck_seconds
overlap = sr * overlap_seconds
hop = sr * hop_seconds

#empty feature matrix
all_feature_matrix = []

#loop over all the files
for f in tqdm(file_ids):

    #make the file location
    file_loc = music_folder + f['file_name']

    print("\nWorking on... " + file_loc)

    #read the audio in librosa
    audio, sr = lr.load(file_loc, sr=sr)
    total_samples = len(audio)
    total_chunks = math.ceil(total_samples/hop)

    #loop for all possible features
    i = 0
    while i < total_chunks:

        #make an empty feature matrix
        feature_matrix = np.zeros((chunk), dtype='float32')

        #get the start and end point for the sample
        start = (i * hop) - overlap
        end = (i * hop) + chunk - overlap


        #[OB] There must be a better way to write this if conditional code
        if start < 0:
            #padding zeros before the first sample
            feature_matrix[(i + overlap):] = audio[0:end]
        elif end > total_samples:
            #helping to pad zeroes in the last sample
            end = total_samples - 1
            feature_matrix[:len(audio[start:end])] = audio[start:end]
        else:
            #add the features   
            feature_matrix = audio[start:end]        

        #append to the actual matrix
        all_feature_matrix.append(feature_matrix)

        #increase i to loop
        i += 1

#convert the list into a numpy array
all_feature_matrix = np.array(all_feature_matrix)
print("\nFeature Extraction Completed!")
print("Shape of the features: ", all_feature_matrix.shape)

#save the numpy array as pickle
np.save("data/feature_data",all_feature_matrix)
print("\nFeature Saved in directory!")

  0%|          | 0/1 [00:00<?, ?it/s]


Working on... full_albums_wav/RussianCircles_BloodYear.wav


100%|██████████| 1/1 [01:23<00:00, 83.45s/it]


Feature Extraction Completed!
Shape of the features:  (265, 880)

Feature Saved in directory!


# 3: Converting the Files into numpy array for the system - Using onset

**Inputs**:

Variable name | Description
--- | ---
chunck_seconds | amount of sound data each chuck should have in it
overlap_seconds | how much overlap is expected with the previous chuck (will consider padding for the first chunk)
sr | sampling rate while reading audio, also how many features per chunk (default: 44100)
hop_seconds | how much seconds to hop for the next chuck (default: starts at the end of previous chunk, considering overlap)

---

**Outputs**:

- Prints the total number of features created as (no. of features/chunks, samples in each chunk)
- saves the features into a file: 'data/feature_data'


In [ ]:
#Folder from where files need to be read
#can be converted to glob later
music_folder = ""


#specifics for the audio extraction
chunck_seconds = 10
overlap_seconds = 1
sr = global_sr
#hop_seconds = 8
hop_seconds = chunck_seconds - overlap_seconds          #default value

#concerting inputs to samples
chunk = sr * chunck_seconds
overlap = sr * overlap_seconds
hop = sr * hop_seconds

#empty feature matrix
all_feature_matrix = []

#loop over all the files
for f in tqdm(file_ids):

    #make the file location
    file_loc = music_folder + f['file_name']

    print("\nWorking on... " + file_loc)

    #read the audio in librosa
    audio, sr = lr.load(file_loc, sr=sr)
    total_samples = len(audio)
    total_chunks = math.ceil(total_samples/hop)

    #loop for all possible features
    i = 0
    while i < total_chunks:

        #make an empty feature matrix
        feature_matrix = np.zeros((chunk), dtype='float32')

        #get the start and end point for the sample
        start = (i * hop) - overlap
        end = (i * hop) + chunk - overlap


        #[OB] There must be a better way to write this if conditional code
        if start < 0:
            #padding zeros before the first sample
            feature_matrix[(i + overlap):] = audio[0:end]
        elif end > total_samples:
            #helping to pad zeroes in the last sample
            end = total_samples - 1
            feature_matrix[:len(audio[start:end])] = audio[start:end]
        else:
            #add the features   
            feature_matrix = audio[start:end]        

        #append to the actual matrix
        all_feature_matrix.append(feature_matrix)

        #increase i to loop
        i += 1

#convert the list into a numpy array
all_feature_matrix = np.array(all_feature_matrix)
print("\nFeature Extraction Completed!")
print("Shape of the features: ", all_feature_matrix.shape)

#save the numpy array as pickle
np.save("data/feature_data",all_feature_matrix)
print("\nFeature Saved in directory!")

  0%|          | 0/1 [00:00<?, ?it/s]


Working on... full_albums_wav/RussianCircles_BloodYear.wav


100%|██████████| 1/1 [01:23<00:00, 83.45s/it]


Feature Extraction Completed!
Shape of the features:  (265, 880)

Feature Saved in directory!
